In [1]:
#pip install seaborn
import warnings
warnings.filterwarnings('ignore')

from pytorch_tabular import TabularModel
from pytorch_tabular.models import GatedAdditiveTreeEnsembleConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    #ExperimentConfig,
)
from pytorch_tabular.utils import get_class_weighted_cross_entropy
#pip install pytorch_tabular[extra]
from evaluation.generalevaluator import *
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, fetch_california_housing, load_breast_cancer
from factory import create_data_loader
import pandas as pd

In [3]:
import os
print(os.getcwd())

/Users/mdicio/Documents/GitHub/WTabRun/notebooks


In [2]:
data_loader = create_data_loader('covertype', test_size=0.2, normalize_features = "mean_std", return_extra_info = True)
X_train, X_val, y_train, y_val, extra_info = data_loader.load_data()

FileNotFoundError: [Errno 2] No such file or directory: './data/covertype/covtype.data.gz'

# GATE Tests

In [3]:
total_count = 891
class_count = [549, 342]
class_weights = [total_count / count for count in class_count]
class_weights

[1.6229508196721312, 2.6052631578947367]

In [4]:
mu = class_weights[0] / class_weights[1] # 0.620
weighted_loss = get_class_weighted_cross_entropy(y_train.values.ravel(), mu=mu)
print(f"mu = {mu}")

mu = 0.6229508196721312


In [5]:
# Define the data configuration

from torchmetrics.regression import MeanSquaredError


data_config = DataConfig(
    target=['target'],
    continuous_cols= [i for i in extra_info["num_col_names"] if i != "target"],
    categorical_cols=extra_info["cat_col_names"],
)
trainer_config = TrainerConfig(
    auto_lr_find = False,  # Runs the LRFinder to automatically derive a learning rate
    batch_size   = 512,
    max_epochs   = 2,
)

optimizer_config = OptimizerConfig()

from torchmetrics.functional.regression import mean_squared_error
model_config = GatedAdditiveTreeEnsembleConfig(
    task="classification",
    tree_depth  =  4,
    num_trees   = 12,
    chain_trees = True, # akin to bagging, True is akin to boosting
    gflu_stages =  2
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-07-27 00:47:48,921 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off


In [6]:
help(TrainerConfig)

Help on class TrainerConfig in module pytorch_tabular.config.config:

class TrainerConfig(builtins.object)
 |  TrainerConfig(batch_size: int = 64, data_aware_init_batch_size: int = 2000, fast_dev_run: bool = False, max_epochs: int = 10, min_epochs: Optional[int] = 1, max_time: Optional[int] = None, gpus: Optional[int] = None, accelerator: Optional[str] = 'auto', devices: Optional[int] = None, devices_list: Optional[List[int]] = None, accumulate_grad_batches: int = 1, auto_lr_find: bool = False, auto_select_gpus: bool = True, check_val_every_n_epoch: int = 1, gradient_clip_val: float = 0.0, overfit_batches: float = 0.0, deterministic: bool = False, profiler: Optional[str] = None, early_stopping: Optional[str] = 'valid_loss', early_stopping_min_delta: float = 0.001, early_stopping_mode: str = 'min', early_stopping_patience: int = 3, early_stopping_kwargs: Optional[Dict[str, Any]] = <factory>, checkpoints: Optional[str] = 'valid_loss', checkpoints_path: str = 'saved_models', checkpoints_e

In [7]:
assert X_train.isnull().sum().sum() == 0

In [8]:
import torch.nn as nn

In [67]:
X_train

,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone,family_size
692,0.829568,male,-1.373384,-0.465084,-0.466183,0.513812,S,Third,NA,Southampton,0.800346,-0.556339
481,-0.370945,male,-1.373384,-0.465084,-0.466183,-0.662563,S,Second,NA,Southampton,0.800346,-0.556339
527,-1.571457,male,-1.373384,-0.465084,-0.466183,3.955399,S,First,C,Southampton,0.800346,-0.556339
855,0.829568,female,-0.346436,-0.465084,0.727782,-0.467874,S,Third,NA,Southampton,-1.249460,0.073412
801,-0.370945,female,0.395248,0.478335,0.727782,-0.115977,S,Second,NA,Southampton,-1.249460,0.703162
...,...,...,...,...,...,...,...,...,...,...,...,...
359,0.829568,female,-1.373384,-0.465084,-0.466183,-0.498500,Q,Third,NA,Queenstown,0.800346,-0.556339
258,-1.571457,female,0.623459,-0.465084,-0.466183,10.005329,C,First,NA,Cherbourg,0.800346,-0.556339
736,0.829568,female,1.365143,0.478335,3.115713,0.053205,S,Third,NA,Southampton,-1.249460,1.962663
462,-1.571457,male,1.308091,-0.465084,-0.466183,0.139097,S,First,E,Southampton,0.800346,-0.556339


In [10]:
validation

,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone,family_size,target
565,0.829568,male,-0.004120,1.421753,-0.466183,-0.159704,S,Third,NA,Southampton,-1.249460,0.703162,0
160,0.829568,male,1.136933,-0.465084,0.727782,-0.327324,S,Third,NA,Southampton,-1.249460,0.073412,0
553,0.829568,male,-0.118226,-0.465084,-0.466183,-0.512122,C,Third,NA,Cherbourg,0.800346,-0.556339,1
860,0.829568,male,0.965775,1.421753,-0.466183,-0.368795,S,Third,NA,Southampton,-1.249460,0.703162,0
241,0.829568,female,-1.373384,0.478335,-0.466183,-0.339817,Q,Third,NA,Queenstown,-1.249460,0.073412,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,-0.370945,female,0.052932,-0.465084,0.727782,-0.121182,S,Second,NA,Southampton,-1.249460,0.073412,1
91,0.829568,male,-0.232331,-0.465084,-0.466183,-0.499020,S,Third,NA,Southampton,0.800346,-0.556339,0
883,-0.370945,male,0.224090,-0.465084,-0.466183,-0.443929,S,Second,NA,Southampton,0.800346,-0.556339,0
473,-0.370945,female,-0.061173,-0.465084,-0.466183,-0.375388,C,Second,D,Cherbourg,0.800346,-0.556339,1


In [9]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# Merge X_train and y_train  
train = pd.concat([X_train, y_train], axis=1)

# Merge X_val and y_val
validation = pd.concat([X_val, y_val], axis=1)

tabular_model.fit(
    train=train, 
    validation=validation,
    loss=weighted_loss
)

2023-07-27 00:48:04,738 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-07-27 00:48:04,776 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-07-27 00:48:04,778 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for classification task
2023-07-27 00:48:04,836 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: GatedAdditiveTreeEnsembleModel
2023-07-27 00:48:04,987 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-07-27 00:48:05,078 - {pytorch_tabular.tabular_model:582} - INFO - Training Started


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ custom_loss      │ CrossEntropyLoss           │      0 │
│ 1 │ _backbone        │ GatedAdditiveTreesBackbone │  111 K │
│ 2 │ _embedding_layer │ Embedding1dLayer           │    103 │
│ 3 │ _head            │ CustomHead                 │     46 │
└───┴──────────────────┴────────────────────────────┴────────┘

Trainable params: 111 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 111 K                                                                                                
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=2` reached.


2023-07-27 00:48:07,666 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-07-27 00:48:07,668 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model


In [70]:
pred_df = tabular_model.predict(validation)

Output()

In [71]:
pred_df

,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone,family_size,target,0_probability,1_probability,prediction
565,0.829568,male,-0.004120,1.421753,-0.466183,-0.159704,S,Third,NA,Southampton,-1.249460,0.703162,0,0.581861,0.418139,0
160,0.829568,male,1.136933,-0.465084,0.727782,-0.327324,S,Third,NA,Southampton,-1.249460,0.073412,0,0.576033,0.423967,0
553,0.829568,male,-0.118226,-0.465084,-0.466183,-0.512122,C,Third,NA,Cherbourg,0.800346,-0.556339,1,0.583989,0.416011,0
860,0.829568,male,0.965775,1.421753,-0.466183,-0.368795,S,Third,NA,Southampton,-1.249460,0.703162,0,0.580584,0.419416,0
241,0.829568,female,-1.373384,0.478335,-0.466183,-0.339817,Q,Third,NA,Queenstown,-1.249460,0.073412,1,0.583330,0.416670,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,-0.370945,female,0.052932,-0.465084,0.727782,-0.121182,S,Second,NA,Southampton,-1.249460,0.073412,1,0.580422,0.419578,0
91,0.829568,male,-0.232331,-0.465084,-0.466183,-0.499020,S,Third,NA,Southampton,0.800346,-0.556339,0,0.583955,0.416045,0
883,-0.370945,male,0.224090,-0.465084,-0.466183,-0.443929,S,Second,NA,Southampton,0.800346,-0.556339,0,0.585967,0.414033,0
473,-0.370945,female,-0.061173,-0.465084,-0.466183,-0.375388,C,Second,D,Cherbourg,0.800346,-0.556339,1,0.588488,0.411512,0


In [22]:
pred_df = tabular_model.predict(validation)
pred_df.head()
# Add PassengerId information to the predictions
pred_df_with_ids = pd.concat([df_val_id.reset_index(drop=True), pred_df.reset_index(drop=True)], axis=1)

pred_df_with_ids = pred_df_with_ids.rename(columns={'prediction': 'Survived'})
# Display the predictions with PassengerId information
pred_df_with_ids[["PassengerId", "Survived"]].head()

Output()

NameError: name 'df_val_id' is not defined

In [ ]:
pred_df_with_ids["Survived"].value_counts()

In [ ]:
pred_df_with_ids[["PassengerId", "Survived"]].to_csv("/Users/mdicio/Desktop/doidownloads/kaggle_datasets/test_submission_boom.csv", index = False)

# TabTransformer Tests

In [ ]:
#pip install tab-transformer-pytorch
import torch
from tab_transformer_pytorch import FTTransformer

In [ ]:

model = FTTransformer(
    categories = (5, 5, 5, 5, 5),      # tuple containing the number of unique values within each category
    num_continuous = 10,                # number of continuous values
    dim = 32,                           # dimension, paper set at 32
    dim_out = 1,                        # binary prediction, but could be anything
    depth = 6,                          # depth, paper recommended 6
    heads = 8,                          # heads, paper recommends 8
    attn_dropout = 0.1,                 # post-attention dropout
    ff_dropout = 0.1                    # feed forward dropout
)

x_categ = torch.randint(0, 5, (1, 5))     # category values, from 0 - max number of categories, in the order as passed into the constructor above
x_numer = torch.randn(1, 10)              # numerical value

pred = model(x_categ, x_numer) # (1, 1)

In [ ]:
x_categ.shape, x_numer.shape

In [ ]:
x_categ = torch.randint(0, 5, (1, 5))  
x_categ, x_numer

In [ ]:
pred

In [ ]:
x

# Tabnet Tests

In [7]:
# Split the train data into training and validation sets
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.callbacks import EarlyStopping

In [10]:
help(TabNetClassifier)

Help on class TabNetClassifier in module pytorch_tabnet.tab_model:

class TabNetClassifier(pytorch_tabnet.abstract_model.TabModel)
 |  TabNetClassifier(n_d: int = 8, n_a: int = 8, n_steps: int = 3, gamma: float = 1.3, cat_idxs: List[int] = <factory>, cat_dims: List[int] = <factory>, cat_emb_dim: int = 1, n_independent: int = 2, n_shared: int = 2, epsilon: float = 1e-15, momentum: float = 0.02, lambda_sparse: float = 0.001, seed: int = 0, clip_value: int = 1, verbose: int = 1, optimizer_fn: Any = <class 'torch.optim.adam.Adam'>, optimizer_params: Dict = <factory>, scheduler_fn: Any = None, scheduler_params: Dict = <factory>, mask_type: str = 'sparsemax', input_dim: int = None, output_dim: int = None, device_name: str = 'auto', n_shared_decoder: int = 1, n_indep_decoder: int = 1) -> None
 |  
 |  Method resolution order:
 |      TabNetClassifier
 |      pytorch_tabnet.abstract_model.TabModel
 |      sklearn.base.BaseEstimator
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |

['self', 'n_d', 'n_a', 'n_steps', 'gamma', 'cat_idxs', 'cat_dims', 'cat_emb_dim', 'n_independent', 'n_shared', 'epsilon', 'momentum', 'lambda_sparse', 'seed', 'clip_value', 'verbose', 'optimizer_fn', 'optimizer_params', 'scheduler_fn', 'scheduler_params', 'mask_type', 'input_dim', 'output_dim', 'device_name', 'n_shared_decoder', 'n_indep_decoder']


In [8]:
# Define the early stopping callback
early_stopping = EarlyStopping(early_stopping_metric = 'val_loss',
is_maximize = False,
patience = 32
)

model = TabNetClassifier()

# Train the model with early stopping
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_name=['valid'],
    eval_metric=['logloss'],
    max_epochs=2,
    batch_size = 16,
    callbacks=[early_stopping],
patience = 32)            

TypeError: Pandas DataFrame are not supported: apply X.values when calling fit

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# Predict the labels of the validation data
y_pred = model.predict(X_test)
# Calculate the score using the specified metric
evaluator = Evaluator(problem_type = "multiclass_classification")
evaluator.y_true = y_val
evaluator.y_pred = y_pred
score = evaluator.evaluate_metric(metric_name = "f1")